<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Exercises" data-toc-modified-id="Exercises-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Exercises</a></span><ul class="toc-item"><li><span><a href="#Query-Exercise-1" data-toc-modified-id="Query-Exercise-1-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Query Exercise 1</a></span></li><li><span><a href="#Query-Exercise-2" data-toc-modified-id="Query-Exercise-2-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Query Exercise 2</a></span></li><li><span><a href="#Query-Exercise-3---Students-to-do-on-their-own" data-toc-modified-id="Query-Exercise-3---Students-to-do-on-their-own-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Query Exercise 3 - Students to do on their own</a></span></li><li><span><a href="#Query-Exercise-4" data-toc-modified-id="Query-Exercise-4-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Query Exercise 4</a></span></li><li><span><a href="#Query-Exercise-5" data-toc-modified-id="Query-Exercise-5-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Query Exercise 5</a></span></li><li><span><a href="#Great!-It's-working.-Let's-add-some-group-by-to-remove-duplicates!" data-toc-modified-id="Great!-It's-working.-Let's-add-some-group-by-to-remove-duplicates!-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Great! It's working. Let's add some group by to remove duplicates!</a></span></li></ul></li></ul></div>

# Exercises

We'll walk through some of these and aim to have you all do them yourself!

In [12]:
def sql_with_cols(query, cursor=cur):
  '''
  Gives me the full result (with columns)
  '''
  result = cursor.execute(query).fetchall()
  cols = tuple([description[0] for description in cursor.description])
  
  full_result = [cols] + result[:] 
  return full_result

In [14]:
# Now we can put these results into a DataFrame
def sql_to_df(query, cursor=cur):
    '''
    Create a DataFrame directly from a SQL query.
    '''
    results = sql_with_cols(query, cursor)
    return pd.DataFrame(columns=results[0], data=results[1:])

## Query Exercise 1

Create the following table using the customers table and the orders table

<img src="images/query1.png" width="500"/>

In [1]:
import pandas as pd
import sqlite3
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

In [2]:
cur.execute(''' SELECT * 
                FROM customers
                LIMIT 2
                ''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",,Nantes,,44000,France,1370,21000.00
1,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,,Las Vegas,NV,83030,USA,1166,71800.00


In [4]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())
q = ('''SELECT * 
        FROM df
        LIMIT 2
        ;''')

In [6]:
results = pysqldf(q)
results.head()

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",,Nantes,,44000,France,1370,21000.00
1,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,,Las Vegas,NV,83030,USA,1166,71800.00


In [9]:
cur.execute(''' SELECT * 
                FROM orders
                LIMIT 2
                ''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber
0,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,,363
1,10101,2003-01-09,2003-01-18,2003-01-11,Shipped,Check on availability.,128


In [10]:
cur.execute('''
            SELECT c.customerName, c.phone, c.state, o.orderDate
            FROM customers as c
            JOIN orders as o
            USING (customerNumber)
            ''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,customerName,phone,state,orderDate
0,Atelier graphique,40.32.2555,,2003-05-20
1,Atelier graphique,40.32.2555,,2004-09-27
2,Atelier graphique,40.32.2555,,2004-11-25
3,Signal Gift Stores,7025551838,NV,2003-05-21
4,Signal Gift Stores,7025551838,NV,2004-08-06


<details>
    <summary>Query 1 - Solution</summary>
    Query:
    
```SQL
SELECT c.customerName, c.phone, c.state, o.orderDate FROM
customers as c
JOIN orders as o using (customerNumber);
```
</details>

## Query Exercise 2

Create a query that results in a table with the every customer number for an employee along with the employee name and number.

Ex:

<img src="images/query2.png" width="500"/>

In [15]:
# Work here
sql_to_df('''
SELECT
    *
From 
    customers
''')

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",,Nantes,,44000,France,1370,21000.00
1,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,,Las Vegas,NV,83030,USA,1166,71800.00
2,114,"Australian Collectors, Co.",Ferguson,Peter,03 9520 4555,636 St Kilda Road,Level 3,Melbourne,Victoria,3004,Australia,1611,117300.00
3,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",,Nantes,,44000,France,1370,118200.00
4,121,Baane Mini Imports,Bergulfsen,Jonas,07-98 9555,Erling Skakkes gate 78,,Stavern,,4110,Norway,1504,81700.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,486,Motor Mint Distributors Inc.,Salazar,Rosa,2155559857,11328 Douglas Av.,,Philadelphia,PA,71270,USA,1323,72600.00
118,487,Signal Collectibles Ltd.,Taylor,Sue,4155554312,2793 Furth Circle,,Brisbane,CA,94217,USA,1165,60300.00
119,489,"Double Decker Gift Stores, Ltd",Smith,Thomas,(171) 555-7555,120 Hanover Sq.,,London,,WA1 1DP,UK,1501,43300.00
120,495,Diecast Collectables,Franco,Valarie,6175552555,6251 Ingle Ln.,,Boston,MA,51003,USA,1188,85100.00


In [16]:
sql_to_df('''
SELECT *
from employees
''')

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle
0,1002,Murphy,Diane,x5800,dmurphy@classicmodelcars.com,1,,President
1,1056,Patterson,Mary,x4611,mpatterso@classicmodelcars.com,1,1002,VP Sales
2,1076,Firrelli,Jeff,x9273,jfirrelli@classicmodelcars.com,1,1002,VP Marketing
3,1088,Patterson,William,x4871,wpatterson@classicmodelcars.com,6,1056,Sales Manager (APAC)
4,1102,Bondur,Gerard,x5408,gbondur@classicmodelcars.com,4,1056,Sale Manager (EMEA)
5,1143,Bow,Anthony,x5428,abow@classicmodelcars.com,1,1056,Sales Manager (NA)
6,1165,Jennings,Leslie,x3291,ljennings@classicmodelcars.com,1,1143,Sales Rep
7,1166,Thompson,Leslie,x4065,lthompson@classicmodelcars.com,1,1143,Sales Rep
8,1188,Firrelli,Julie,x2173,jfirrelli@classicmodelcars.com,2,1143,Sales Rep
9,1216,Patterson,Steve,x4334,spatterson@classicmodelcars.com,2,1143,Sales Rep


In [17]:
query = """
SELECT
    e.firstName
    ,e.lastName
    ,e.employeeNumber
    ,c.customerNumber
FROM
    employees as e
    join customers as c
        on e.employeeNumber = c.salesRepEmployeeNumber
        """

# pd.read_sql(query, conn).head()
sql_to_df(query).head()

,firstName,lastName,employeeNumber,customerNumber
0,Leslie,Jennings,1165,124
1,Leslie,Jennings,1165,129
2,Leslie,Jennings,1165,161
3,Leslie,Jennings,1165,321
4,Leslie,Jennings,1165,450


<details>
    <summary> Query 2 - Solution</summary>
    Query:
    
```SQL
select e.firstname, e.lastname, e.employeenumber, c.customerNumber
from employees as e
join customers as c
on e.employeenumber = c.salesrepemployeenumber;
```
    
</details>

In [18]:
# Now let's take this and incorporate the next table orders
orders_df.head(1)

NameError: name 'orders_df' is not defined

## Query Exercise 3 - Students to do on their own

Create a query that results in a table with every customer name, order date, where status is disputed (you will need the customers and orders table to complete this task)

Ex:

<img src="images/query_3.png" width="500"/>

In [ ]:
# Work here

In [ ]:
query = """
        SELECT
        """

pd.read_sql(query, conn).head()

<details>
    <summary> Query 3 - Solution</summary>
    Query:
    
```SQL
SELECT c.customerName, o.status, o.orderDate
FROM customers c
JOIN orders o
USING (customerNumber)
WHERE status = "Disputed"
```
    
</details>

## Query Exercise 4

Write a query that results in the following:

Employee First and Last Name, order number and each product code for that order. 

*There should be a row for each product code*

Screenshot:

<img src="images/query4.png" width="500"/>

In [ ]:
# work here
query = """
        select
        """

pd.read_sql(query, conn).head(2)

<details>
    <summary>Query 4 Solution</summary>
    
    Query:
    
```SQL
    select e.firstname, e.lastname, o.orderNumber, od.productCode
    from employees as e
    join customers as c
    on e.employeenumber = c.salesrepemployeenumber
    join orders as o
    on o.customerNumber = c.customerNumber
    join orderdetails as od
    on od.orderNumber = o.orderNumber;
```
        
</details>

## Query Exercise 5

Write a query that results in the following:

Employee First and Last Name and each product name they sold, add an alias to match the image below

*There should be a row for each product name*

Screenshot:

<img src="images/query5.png" width="500"/>

In [ ]:
# work here
query = """
        select
        """

pd.read_sql(query, conn).head(2)

<details>
    <summary>Query 5 Solution</summary>
    
    Query:
    
```SQL
select e.firstname, e.lastname, p.productName
from employees as e
join customers as c
on e.employeenumber = c.salesrepemployeenumber
join orders as o
on o.customerNumber = c.customerNumber
join orderdetails as od
on od.orderNumber = o.orderNumber
join products as p
on p.productCode = od.productCode;
```
        
</details>

## Great! It's working. Let's add some group by to remove duplicates!

**Add aliases to result in the following**

<img src="images/query5.png" width="500"/>

In [ ]:
# remember the alias' from above

query = '''SELECT ...'''

final_df = pd.read_sql(query, conn)
final_df.head(2)

<details>
<summary>Final Query Solution</summary>

    Query:
    
```SQL
select e.firstname as fn, e.lastname as ln, p.productName as pn
from employees as e
join customers as c
on e.employeenumber = c.salesrepemployeenumber
join orders as o
on o.customerNumber = c.customerNumber
join orderdetails as od
on od.orderNumber = o.orderNumber
join products as p
on p.productCode = od.productCode
group by fn, ln, pn;

```

</details>